## VQGAN+CLIP (Text-to-Image)

A machine dreams about text. Combines OpenAI's [CLIP](https://openai.com/blog/clip/) which is able to map images to natural language with the recent image synthesis model [Taming Transformers](https://compvis.github.io/taming-transformers/). The image improves iteratively.




In [ ]:
#@title Inputs
output_path = "/content/output"
text_input = 'Vibrant painting of a movie in the style of Magritte. '  #@param {type: "string"}
text_not = 'disconnected, confusing, incoherent, cluttered, watermarks, text, writing'  #@param {type: "string"}
num_iterations = 700  #@param {type: "number"}
num_save_every = 10 #@param {type: "number"}

In [ ]:
#@title Get Text Slug
!rm $output_path/*.png $output_path/*.mp4
!mkdir -p $output_path

!pip install slugify
from slugify import slugify
text_slug = slugify(text_input)[:200]

rm: cannot remove '/content/output/*.png': No such file or directory
rm: cannot remove '/content/output/*.mp4': No such file or directory
  Created wheel for slugify: filename=slugify-0.0.1-cp37-none-any.whl size=1909 sha256=8f1a4b72ad76bed5ce11dc726e19d29b3f0687d77b0bd0eba3a248fc60124b2c
  Stored in directory: /root/.cache/pip/wheels/f9/68/7b/b797d07992b6c7ef1666a59cc79f45fb37aab4344efa6cc3e1
Successfully built slugify


In [ ]:
#@title Check GPU Type
# were you lucky today?

!nvidia-smi -L


GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-54a85d01-1f18-d430-e31b-6013c1789ae7)


# Top (imports)


In [ ]:
#@title Import Libraries

# don't use half of these lol
!pip install einops
import torch
import numpy as np
import torchvision
import torchvision.transforms.functional as TF
!pip install kornia
import kornia

import PIL
import matplotlib.pyplot as plt

import os
import random
import imageio
from IPython import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import glob

from google.colab import output



     |████████████████████████████████| 296kB 7.4MB/s 
     |████████████████████████████████| 14.6MB 369kB/s 
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.19.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


/usr/local/lib/python3.7/dist-packages/kornia/augmentation/augmentation.py:1875: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


# Perceptor

In [ ]:
#@title Initialize CLIP Perceptor
!pip install --no-deps ftfy regex tqdm
!git clone https://github.com/openai/CLIP.git

#Modify CLIP to output patches
# import fileinput
# with fileinput.FileInput('CLIP/clip/model.py', inplace=True, backup='.bak') as file:
#     for line in file:
#         print(line.replace("x = self.ln_post(x[:, 0, :])", "x = self.ln_post(x[:, :, :])"), end='')

#Import CLIP and load the model
from CLIP import clip
perceptor, preprocess = clip.load('ViT-B/32', jit=False)
perceptor.eval()

clip.available_models()

perceptor.visual.input_resolution

scaler = 1
# perceptor.visual.scale = 224*scaler
# # perceptor.visual.positional_embedding = torch.nn.Parameter(torch.cat([perceptor.visual.positional_embedding]*5 + [perceptor.visual.positional_embedding[:7]]))



# # pos_ed = []
# # for p in range(1, len(perceptor.visual.positional_embedding)):
# #   # pos_ed.append((perceptor.visual.positional_embedding.data[p//4] + perceptor.visual.positional_embedding.data[p//4 + 1]) / 2)
# #   pos_ed.append((perceptor.visual.positional_embedding.data[[p]]))
# #   pos_ed.append((perceptor.visual.positional_embedding.data[[p]]))
# #   pos_ed.append((perceptor.visual.positional_embedding.data[[p]]))
# #   pos_ed.append((perceptor.visual.positional_embedding.data[[p]]))

# # perceptor.visual.positional_embedding.shape

# thems = torchvision.transforms.Resize((scaler*7, scaler*7), interpolation=torchvision.transforms.InterpolationMode.BILINEAR)(perceptor.visual.positional_embedding.data[1:].T.view(1, 768, 7, 7))

# # thems = perceptor.visual.positional_embedding.data[1:].T.repeat_interleave(scaler**2).view(1, 768, 7*scaler, 7*scaler)

# thems = thems.permute(0, 2, 3, 1).view(-1, 768)

# perceptor.visual.positional_embedding = torch.nn.Parameter(torch.cat([perceptor.visual.positional_embedding.data[:1], thems]))

# # perceptor.visual.positional_embedding.shape



     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=e5dff286ae95c1e416e2d2933629b5a24723e7b4fd06fb226a04671de4939960
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy
Cloning into 'CLIP'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.


100%|███████████████████████████████████████| 354M/354M [00:05<00:00, 59.5MiB/s]
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


CLIP(
  (visual): VisualTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

['RN50', 'RN101', 'RN50x4', 'ViT-B/32']

224

# Params

In [ ]:
#@title Define Image Parameters
im_shape = [512, 512, 3]
sideX, sideY, channels = im_shape
batch_size = 1


# Define

In [ ]:
#@title Define Helpers
import os
def displ(img, pre_scaled=True, alternative=False):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1, 2, 0))
  if not pre_scaled:
    img = scale(img, 48*4, 32*4)
  #if os.path.exists(outpath(itt)):
  if itt >= 10:
    imageio.imwrite(outpath(itt,alternative=alternative), np.array(img))
  else:
    print("skipping image writing for iteration", itt)
  #if itt%10==0:
  #  return display.Image(outpath(itt,alternative=alternative))

def gallery(array, ncols=2):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    # want result.shape = (height*nrows, width*ncols, intensity)
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

def card_padded(im, to_pad=3):
  return np.pad(np.pad(np.pad(im, [[1,1], [1,1], [0,0]],constant_values=0), [[2,2], [2,2], [0,0]],constant_values=1),
            [[to_pad,to_pad], [to_pad,to_pad], [0,0]],constant_values=0)

def get_all(img):
  img = np.transpose(img, (0,2,3,1))
  cards = np.zeros((img.shape[0], sideX+12, sideY+12, 3))
  for i in range(len(img)):
    cards[i] = card_padded(img[i])
  print(img.shape)
  cards = gallery(cards)
  imageio.imwrite(outpath(itt), np.array(cards))
  if itt % 30 == 0:
    return display.Image(outpath(itt))
  

# Generator

In [ ]:
#@title DownLoad VQGan Model
!pip uninstall torchtext --yes

%cd /content/
!git clone https://github.com/CompVis/taming-transformers  
%cd /content/taming-transformers


# download a VQGAN with a larger codebook (16384 entries)
!mkdir -p logs/vqgan_imagenet_f16_16384/checkpoints
!mkdir -p logs/vqgan_imagenet_f16_16384/configs

if len(os.listdir('logs/vqgan_imagenet_f16_16384/checkpoints/')) == 0:
  !wget 'https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1' -O 'logs/vqgan_imagenet_f16_16384/checkpoints/last.ckpt' 
  !wget 'https://heibox.uni-heidelberg.de/f/274fb24ed38341bfa753/?dl=1' -O 'logs/vqgan_imagenet_f16_16384/configs/model.yaml' 



# !cp /content/drive/MyDrive/vqgan_imagenet_f16_16384-20210325T002625Z-001.zip /content/vq.zip
# !unzip /content/vq.zip -d /content/taming-transformers/logs/


%pip install omegaconf==2.0.0 pytorch-lightning==1.0.8
import sys
sys.path.append(".")


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


import yaml
import torch
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None):
  model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()

def preprocess_vqgan(x):
  x = 2.*x - 1.
  return x

def custom_to_pil(x):
  x = x.detach().cpu()
  x = torch.clamp(x, -1., 1.)
  x = (x + 1.)/2.
  x = x.permute(1,2,0).numpy()
  x = (255*x).astype(np.uint8)
  x = Image.fromarray(x)
  if not x.mode == "RGB":
    x = x.convert("RGB")
  return x

def reconstruct_with_vqgan(x, model):
  # could also use model(x) for reconstruction but use explicit encoding and decoding here
  z, _, [_, _, indices] = model.encode(x)
  print(f"VQGAN: latent shape: {z.shape[2:]}")
  xrec = model.decode(z)
  return xrec

config16384 = load_config("logs/vqgan_imagenet_f16_16384/configs/model.yaml", display=False)
model16384 = load_vqgan(config16384, ckpt_path="logs/vqgan_imagenet_f16_16384/checkpoints/last.ckpt").to(DEVICE)



Uninstalling torchtext-0.10.0:
  Successfully uninstalled torchtext-0.10.0
/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 756, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 756 (delta 59), reused 50 (delta 50), pack-reused 683
Receiving objects: 100% (756/756), 174.04 MiB | 33.76 MiB/s, done.
Resolving deltas: 100% (198/198), done.
/content/taming-transformers
--2021-07-03 14:15:38--  https://heibox.uni-heidelberg.de/f/867b05fc8c4841768640/?dl=1
Resolving heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)... 129.206.7.113
Connecting to heibox.uni-heidelberg.de (heibox.uni-heidelberg.de)|129.206.7.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://heibox.uni-heidelberg.de/seafhttp/files/daceea37-955e-4d19-a591-42461dd9e911/last.ckpt [following]
--2021-07-03 14:15:38--  https://heibox.uni-heidelberg.de/seafhttp/files/daceea37-955e-4d19-a591-42461dd9e9

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):


Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


8.19kB [00:00, 359kB/s]                    


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


 # Latent coordinate & Text

In [ ]:
#@title Calculate Latent coordinate & Text
torch.cuda.empty_cache()



class Pars(torch.nn.Module):
    def __init__(self):
        super(Pars, self).__init__()


        # self.normu = torch.nn.Parameter(o_i2.cuda().clone())

        self.normu = .5*torch.randn(batch_size, 256, sideX//16, sideY//16).cuda()
        
        self.normu = torch.nn.Parameter(torch.sinh(1.9*torch.arcsinh(self.normu)))

    def forward(self):


      return self.normu.clip(-6, 6)
      

def model(x):
  # o_i1 = model16384.encoder(x)
  # o_i1 = x
  # o_i2 = model16384.quant_conv(o_i1)
  o_i2 = x
  o_i3 = model16384.post_quant_conv(o_i2)
  i = model16384.decoder(o_i3)
  return i


dec = .1

lats = Pars().cuda()
mapper = [lats.normu]
optimizer = torch.optim.AdamW([{'params': mapper, 'lr': .5}], weight_decay=dec)
eps = 0


tx = clip.tokenize(text_input)
t = perceptor.encode_text(tx.cuda()).detach().clone()

t_not = clip.tokenize(text_not)
t_not = perceptor.encode_text(t_not.cuda()).detach().clone()


nom = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

# augs = torch.nn.Sequential(
#     kornia.augmentation.RandomHorizontalFlip(),
#     kornia.augmentation.ColorJitter(hue=.01, saturation=.01, p=.7),
#     kornia.augmentation.RandomAffine(degrees=15, translate=.05, p=.8, padding_mode='zeros', ),
#     # kornia.augmentation.RandomResizedCrop((int(scaler)*224, int(scaler)*224), (.8, 2), (.8, 1.2)),
    
#     ).cuda()

augs = torch.nn.Sequential(
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomAffine(24, (.1, .1), fill=0)
).cuda()


up_noise = .11

def outpath(itt, ext="png", alternative=False):
  out_path = text_slug+"_"+str(format(itt,'05d')) + '.' + ext
  #if alternative:
  #  out_path = "alternative_"+out_path
  print("out_path", output_path + "/" + out_path)
  return output_path + "/" + out_path

itt = 0


with torch.no_grad():
  al = (model(lats()).cpu().clip(-1, 1) + 1) / 2
  #for allls in al:
  #  displ(allls[:3])
  #  print('\n')
    

# Train

In [ ]:
#@title Training

def augment(into, cutn=32):

  into = torch.nn.functional.pad(into, (sideX//2, sideX//2, sideX//2, sideX//2), mode='constant', value=0)


  into = augs(into)

  p_s = []
  for ch in range(cutn):
    # size = torch.randint(int(.5*sideX), int(1.9*sideX), ())
    size = int(torch.normal(1.2, .3, ()).clip(.43, 1.9) * sideX)
    
    if ch > cutn - 4:
      size = int(sideX*1.4)
    offsetx = torch.randint(0, int(sideX*2 - size), ())
    offsety = torch.randint(0, int(sideX*2 - size), ())
    apper = into[:, :, offsetx:offsetx + size, offsety:offsety + size]
    apper = torch.nn.functional.interpolate(apper, (int(224*scaler), int(224*scaler)), mode='bilinear', align_corners=True)
    p_s.append(apper)
  into = torch.cat(p_s, 0)

  into = into + up_noise*torch.rand((into.shape[0], 1, 1, 1)).cuda()*torch.randn_like(into, requires_grad=False)

  return into

def checkin(loss):
  global up_noise
  
  with torch.no_grad():
    
    alnot = model(lats()).float()
    alnot = augment((((alnot).clip(-1, 1) + 1) / 2), cutn=1)
    for allls in alnot.cpu():
      displ(allls)
      #if itt % 100 == 0:
      #  display.display(display.Image(outpath(itt)))
      print('\n')


    print('''
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    ''')


    alnot = (model(lats()).cpu().clip(-1, 1) + 1) / 2
    
    for allls in alnot.cpu():
      displ(allls,True) #[:, int(.1*sideY):int(.9*sideY), int(.1*sideX):int(.9*sideX)]
      #display.display(display.Image(outpath(itt,alternative=True)))
      print('\n')

  
  print('''
  ##########################################################
  ''',
        loss, ' loss\niteration',itt)


  # the people spoke and they love "ding"
  # output.eval_js('new Audio("https://freesound.org/data/previews/80/80921_1022651-lq.ogg").play()')



def ascend_txt():
  global up_noise
  out = model(lats())


  into = augment((out.clip(-1, 1) + 1) / 2)



  into = nom(into)


  iii = perceptor.encode_image(into)

  t_x = torch.cosine_similarity(t_not, iii, -1)

  all_s = torch.cosine_similarity(t, iii, -1)

  return [3*t_x, -10*all_s]
  
def train(i):
  global dec
  global up_noise

  loss1 = ascend_txt()
  loss = loss1[0] + loss1[1]
  loss = loss.mean()
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  



  if itt > 400:
    for g in optimizer.param_groups:
      g['lr'] *= .998
      g['lr'] = max(g['lr'], .05)
    dec *= .995



  if torch.abs(lats()).max() > 5:
    for g in optimizer.param_groups:
      g['weight_decay'] = dec
  else:
    for g in optimizer.param_groups:
      g['weight_decay'] = 0
  
  if itt % num_save_every == 0:
    checkin(loss1)
    
    print('iteration',itt,'up_noise', up_noise)
    for g in optimizer.param_groups:
      print(g['lr'], 'lr', g['weight_decay'], 'decay')


def loop():
  global itt
  for asatreat in range(num_iterations):
    train(itt)
    itt+=1


#!rm $output_path/*

loop()


skipping image writing for iteration 0



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
skipping image writing for iteration 0



  ##########################################################
   [tensor([0.7837, 0.7773, 0.7217, 0.7822, 0.5840, 0.7349, 0.7725, 0.7661, 0.7891,
        0.7896, 0.6611, 0.6924, 0.7485, 0.7754, 0.6553, 0.8037, 0.7656, 0.7920,
        0.7754, 0.7256, 0.8105, 0.7910, 0.7749, 0.7979, 0.7695, 0.7852, 0.7100,
        0.7617, 0.7803, 0.6743, 0.7354, 0.7695], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-1.9346, -1.9531, -1.9082, -1.9668, -1.2129, -1.6973, -1.8311, -1.9873,
        -1.9785, -2.0195, -1.5635, -1.7598, -1.9375, -1.9727, -1.4834, -1.9199,
        -2.0156, -2.0371, -2.0215, -1.7129, -1.9980, -1.9590, -1.9727, -1.9316,
        -1.9941, -2.0469, -1.5391, -1.8408, -1.9609, -1.7148, -1.8105, -1.8574],
       dev

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00010.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    


out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00010.png



  ##########################################################
   [tensor([0.6255, 0.7969, 0.6670, 0.6992, 0.6025, 0.6201, 0.6484, 0.7129, 0.6851,
        0.6113, 0.6060, 0.7749, 0.6660, 0.6323, 0.6772, 0.6313, 0.6226, 0.6577,
        0.6748, 0.6206, 0.6338, 0.6108, 0.6138, 0.6904, 0.6553, 0.6353, 0.6245,
        0.7031, 0.7041, 0.5928, 0.5664, 0.6289], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-3.3809, -2.4688, -2.9199, -2.8125, -3.5000, -3.4531, -3.4102, -3.0195,
        -2.8125, -3.4570, -3.3867, -2.1992, -2.7266, -3.0078, -3.1738, -3.2227,
        -3.4141, -3.2793, -2.5859, -3.3398, -3.3027, -3.4160, -3.4102, -3.2578,
        -2.7129, -3.1348, -3.4297, -2.3691, -2.9375, -3.4258, -3.4414, -3.2441],
       device='cuda:0', dtype=torch.float16, grad_fn=<MulBackward0>)]  loss
iteration 10
iteration 10 up_noise 0.11
0.5 lr 0.1 decay


out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00020.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00020.png



  ##########################################################
   [tensor([0.5942, 0.6040, 0.5747, 0.6362, 0.5981, 0.6426, 0.5703, 0.7197, 0.6260,
        0.6802, 0.5986, 0.6445, 0.7280, 0.5811, 0.6733, 0.6079, 0.6240, 0.5938,
        0.6055, 0.7417, 0.6670, 0.5742, 0.5708, 0.5986, 0.6895, 0.6230, 0.6465,
        0.6055, 0.5557, 0.5825, 0.5889, 0.5728], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-3.9180, -3.9531, -3.8867, -3.7891, -3.7266, -3.6602, -3.8906, -2.5039,
        -3.3867, -3.2207, -3.8906, -3.5703, -2.7461, -3.8359, -2.8516, -3.7988,
        -3.4766, -3.7910, -3.8770, -2.1836, -3.2656, -3.9570, -3.9941, -3

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00030.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00030.png



  ##########################################################
   [tensor([0.6968, 0.6201, 0.6411, 0.6572, 0.6626, 0.5806, 0.6055, 0.6606, 0.5820,
        0.6221, 0.7661, 0.6284, 0.7383, 0.5898, 0.6772, 0.6475, 0.7041, 0.5732,
        0.7515, 0.5947, 0.6348, 0.6538, 0.6841, 0.6841, 0.6484, 0.6997, 0.5938,
        0.6128, 0.8008, 0.5972, 0.5986, 0.5947], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-3.4531, -4.0586, -3.6914, -3.6016, -4.0664, -3.7852, -3.8945, -4.0234,
        -3.8105, -3.8965, -3.2109, -4.0547, -3.3965, -4.0469, -3.8066, -3.6816,
        -3.1270, -3.8164, -3.3047, -4.0508, -4.0312, -3.9414, -3.5234, -3

...

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00650.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00650.png



  ##########################################################
   [tensor([0.4717, 0.5220, 0.5342, 0.4624, 0.6079, 0.5303, 0.4863, 0.4868, 0.5430,
        0.6167, 0.4988, 0.5869, 0.5928, 0.5068, 0.5693, 0.5767, 0.5806, 0.5957,
        0.5542, 0.5791, 0.5332, 0.6704, 0.5117, 0.5928, 0.5542, 0.5776, 0.5840,
        0.5781, 0.5000, 0.4805, 0.5488, 0.5083], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-5.2500, -4.5469, -3.8457, -4.7695, -4.3008, -5.0781, -5.3477, -5.1719,
        -4.9453, -4.7344, -5.2773, -3.0703, -4.8281, -5.2031, -5.2305, -4.7656,
        -3.7793, -4.1133, -3.7598, -5.0430, -5.0703, -3.8828, -5.1758, -4

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00660.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00660.png



  ##########################################################
   [tensor([0.5640, 0.5273, 0.5190, 0.6450, 0.5566, 0.5576, 0.5532, 0.5693, 0.5137,
        0.5991, 0.5483, 0.5020, 0.6841, 0.6748, 0.6025, 0.5708, 0.7295, 0.5498,
        0.5439, 0.6211, 0.5854, 0.5576, 0.5220, 0.5674, 0.5659, 0.5815, 0.5615,
        0.5566, 0.4546, 0.5205, 0.5869, 0.5703], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-4.9141, -4.9766, -5.2422, -3.8008, -5.1562, -4.4609, -5.1602, -5.1172,
        -4.2305, -4.9883, -5.0352, -5.4102, -3.2773, -2.4629, -4.4180, -5.2344,
        -2.3691, -4.5742, -5.1484, -2.9180, -4.2930, -4.1836, -4.7617, -5

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00670.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00670.png



  ##########################################################
   [tensor([0.6201, 0.5552, 0.6221, 0.5527, 0.7617, 0.5850, 0.4980, 0.5688, 0.6426,
        0.5483, 0.6045, 0.5093, 0.5552, 0.5571, 0.5371, 0.4805, 0.6963, 0.7637,
        0.5205, 0.4790, 0.5439, 0.5303, 0.5322, 0.5518, 0.5522, 0.5112, 0.5576,
        0.6084, 0.5742, 0.5439, 0.4829, 0.5269], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-3.9160, -5.1758, -4.2500, -5.2500, -2.5352, -4.3828, -4.6094, -4.6953,
        -4.2266, -4.5156, -4.5312, -4.9766, -5.2188, -5.2344, -5.3281, -5.0820,
        -2.2617, -2.2188, -5.4297, -5.1875, -5.3438, -5.2305, -5.0391, -4

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00680.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00680.png



  ##########################################################
   [tensor([0.5225, 0.5986, 0.5273, 0.5825, 0.6460, 0.5811, 0.5181, 0.5840, 0.5723,
        0.5591, 0.5664, 0.5791, 0.6113, 0.5796, 0.5811, 0.5894, 0.5498, 0.5522,
        0.5728, 0.5742, 0.6025, 0.6128, 0.5679, 0.5498, 0.5635, 0.5439, 0.5732,
        0.5107, 0.6104, 0.5317, 0.5459, 0.5850], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-5.3750, -5.2031, -5.2891, -3.9258, -4.7695, -4.8281, -4.9570, -4.8828,
        -4.9297, -5.2148, -5.4258, -5.0430, -4.3125, -4.4492, -4.9492, -5.0430,
        -5.0078, -5.0781, -4.9961, -5.3984, -5.1523, -3.9297, -4.2891, -5

out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00690.png



    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    
out_path /content/output/vibrant-painting-of-a-movie-in-the-style-of-magritte_00690.png



  ##########################################################
   [tensor([0.5059, 0.5176, 0.5308, 0.6694, 0.4893, 0.6094, 0.5293, 0.5586, 0.5435,
        0.5508, 0.5352, 0.4756, 0.4695, 0.5757, 0.5918, 0.5527, 0.5439, 0.5439,
        0.5269, 0.5117, 0.6646, 0.7056, 0.5376, 0.5547, 0.6001, 0.4827, 0.4932,
        0.5757, 0.5483, 0.5986, 0.5063, 0.4463], device='cuda:0',
       dtype=torch.float16, grad_fn=<MulBackward0>), tensor([-4.6758, -5.1719, -5.2266, -3.8887, -5.2148, -4.3867, -4.8789, -4.8008,
        -5.0938, -5.1719, -5.0547, -4.4180, -5.1484, -5.0820, -4.9766, -5.3477,
        -3.8516, -4.8359, -5.1484, -4.8750, -3.6406, -3.9453, -4.6445, -4

In [ ]:
#@title Render Video
print(text_slug)

out_file=output_path+"/video_"+text_slug+"_video.mp4"
#!rm /content/*.mp4
!ffmpeg -r 10 -i $output_path/%*.png -y -vf format=yuv420p -vcodec libx264 -acodec aac "$out_file"
#!cp -v /tmp/video.mp4 "$out_file"
#!rm /content/taming-transformers/*.png
print("Written", out_file)
!sleep 2

#from google.colab import files
#files.download(out_file)


vibrant-painting-of-a-movie-in-the-style-of-magritte
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-l

In [ ]:
#@title Render Video with increasing gap (unused)
from glob import glob

# Creates a video from a folder containing images of training progress
# Allows sampling the image less often as iterations increase
# (Requires images be sortable by filename for now. Could use modification date too)
def render_video(output_path="/content/taming-transformers/output.mp4", src_path="/content/taming-transformers/", file_glob="*.png", step_increase_every=100):
  tmp_path = "/tmp/latentvisions_tmp"

  files = glob(src_path+file_glob)
  files.sort()

  if len(files) == 0:
    print(f"No files found in '{src_path}{file_glob}'. Returning...")
    return

  !rm -rf $tmp_path
  !mkdir -p $tmp_path

  for i, filepath in enumerate(files):

    # this is a bit hacky. we should be adding a float to get
    # smooth increase in steps.
    save_step = i % (i//step_increase_every + 1) == 0
    
    if save_step:
      !cp -v $filepath $tmp_path
    else:
      !mv $filepath /tmp

  !ffmpeg -y -r 20 -pattern_type glob -i "{tmp_path+"/"+file_glob}" -vcodec libx264 -crf 15 -pix_fmt yuv420p {output_path} -hide_banner -loglevel error

#render_video(out_file)

#### Thanks!

Many thanks to OpenAI for releasing their models CLIP and DALL-E (the encoder and decoder parts, specifically). I am not affiliated with them.


https://github.com/openai/DALL-E/ (Aditya Ramesh, Mikhail Pavlov, Gabriel Goh, Scott Gray, Chelsea Voss, Alec Radford, Mark Chen, Ilya Sutskever)

https://github.com/openai/CLIP (Alec Radford, \* Jong Wook Kim,\* Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal,
Girish Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever)

Thanks to to Patrick Esser\* Robin Rombach\* Bjorn Ommer for the VQGAN in Taming Transformers! https://github.com/CompVis/taming-transformers


Also, as a good launching point for future directions and to find more related work, see https://distill.pub/2017/feature-visualization/ by Chris Olah, Alexander Mordvintsev, Ludwig Schubert.


\* equal contribution

TopK GAN influenced my choice to screen image patches by coherence. https://arxiv.org/abs/2002.06224 The authors are Samarth Sinha, Zhengli Zhao, Anirudh Goyal, Colin Raffel, and Augustus Odena.

---


Also massive thanks to my patrons and contributors on the Discord who support me. You'll also see that alstroemeria313 (https://twitter.com/RiversHaveWings), CobaltOwl, and Johanezz have contributed the suggestions of weight decay and random-encoded images specifically.

Thanks to ZoeOzone for the title of this notebook: https://twitter.com/zoebot_zoe

Thanks to hotgrits for loading CLIP without restarting. http://torridgristle.tumblr.com/


